<a href="https://colab.research.google.com/github/ndegwaanth/Diabetes_patient_Detection/blob/main/Crop_Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from PIL import Image

In [3]:
# Step 1: Load Dataset
DATASET_PATH = "train/images/"
LABELS_PATH = "train/labels/"
IMG_SIZE = 224

# Get categories from label files
CATEGORIES = sorted(set(f.replace(".txt", "") for f in os.listdir(LABELS_PATH)))

FileNotFoundError: [Errno 2] No such file or directory: 'train/labels/'

In [2]:
def is_image_corrupted(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()  # Verify that the file is a valid image
            img.load()    # Load the image to check for further issues
        return False
    except Exception as e:
        print(f"Error reading {image_path}: {e}")
        return True

def remove_corrupted_images(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            file_path = os.path.join(root, file)
            if is_image_corrupted(file_path):
                print(f"Removing corrupted file: {file_path}")
                os.remove(file_path)

# Path to your dataset
dataset_folder = DATASET_PATH

# Remove corrupted images
remove_corrupted_images(dataset_folder)

NameError: name 'DATASET_PATH' is not defined

In [ ]:
# Step 2: Preprocess Data
X, y = [], []

def load_label(label_path):
    try:
        with open(label_path, "r") as file:
            label = file.readline().strip()
            return CATEGORIES.index(label) if label in CATEGORIES else None
    except Exception as e:
        print(f"Error reading label file {label_path}: {e}")
        return None

for img_name in tqdm(os.listdir(DATASET_PATH), desc="Processing Images"):
    img_path = os.path.join(DATASET_PATH, img_name)
    label_path = os.path.join(LABELS_PATH, img_name.replace(".jpg", ".txt"))

    if not os.path.exists(label_path):
        print(f"No label found for {img_name}, skipping...")
        continue

    label = load_label(label_path)
    if label is None:
        continue

    try:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0  # Normalize
        X.append(img)
        y.append(label)
    except Exception as e:
        print(f"Error processing {img_path}: {e}")

X = np.array(X)
y = np.array(y)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 3: Split Dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
print(f"Training data: {X_train.shape}, Labels: {y_train.shape}")
print(f"Validation data: {X_val.shape}, Labels: {y_val.shape}")
print(f"Test data: {X_test.shape}, Labels: {y_test.shape}")

In [ ]:
# Step 4: Build CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')  # Output layer
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 5: Train the Model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

In [ ]:
# Step 6: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Step 7: Predict on New Images
def predict_image(image_path, model, class_labels):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    return class_labels[class_index]

class_labels = CATEGORIES  # Use label file names as labels
image_path = "test/images/sample.jpg"
predicted_class = predict_image(image_path, model, class_labels)
print(f"Predicted Skin Disease: {predicted_class}")

In [ ]:
# Step 8: Save and Load the Model
model.save("skin_disease_cnn.h5")

In [ ]:
# Load model
model = tf.keras.models.load_model("skin_disease_cnn.h5")